# CH03 LLM,RAG 개인 필수 과제

### 01_API key_load_환경변수(environment_variables)사용

In [2]:
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv('/home/pro/VS/key.env')
r"C:\Users\241011\Documents\key.env"
print('key 활성화 완료')

key 활성화 완료


### 02_PDF loader

In [20]:
import re
import fitz  # PyMuPDF
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from typing import List

# PDF 파일 로드 (PyMuPDF 사용)
def extract_text_from_pdf(pdf_path: str) -> List[str]:
    """PyMuPDF를 사용하여 PDF에서 각 페이지의 텍스트를 추출"""
    doc = fitz.open(pdf_path)
    text_pages = []
    
    # 각 페이지에서 텍스트 추출
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        text_pages.append(text)
    
    return text_pages

# 줄바꿈 제거 함수
def remove_newlines_except_after_period(text: str) -> str:
    """마침표 다음의 줄바꿈을 제외한 모든 줄바꿈을 제거"""
    return re.sub(r'(?<!\.)(\n|\r\n)', ' ', text)

# 연속된 '·' 문자와 숫자 뒤 텍스트 제거하는 함수
def remove_page_numbers_and_bullet_points(text: str) -> str:
    """페이지 번호와 구분 기호 ···· 등 제거"""
    # 숫자, 특수문자(·) 뒤에 이어지는 텍스트를 제거
    text = re.sub(r'\d+\s*▹\s*[\u25CB\u2022\u2023]*\s*\d+', '', text)  # 페이지 번호나 번호 뒤 '·' 제거
    text = re.sub(r'·+', '', text)  # 연속된 '·' 제거
    return text

# 페이지 처리 함수
def process_pdf(pdf_path: str) -> List[Document]:
    pages_text = extract_text_from_pdf(pdf_path)  # PDF에서 텍스트 추출
    
    processed_documents = []
    
    for page_text in pages_text:
        # 텍스트에서 불필요한 줄바꿈 제거
        cleaned_text = remove_newlines_except_after_period(page_text)
        
        # 페이지 번호와 구분 기호 제거
        cleaned_text = remove_page_numbers_and_bullet_points(cleaned_text)
        
        # 처리된 페이지 정보를 새로운 Document로 저장
        processed_documents.append(Document(page_content=cleaned_text, metadata={}))
    
    return processed_documents

# PDF 파일 처리
pdf_path = "00_data/인공지능산업최신동향_2024년11월호.pdf"
pages = process_pdf(pdf_path)

# 결과 출력 (예시)
for doc in pages:
    print(doc.page_content[90:130])  # 처리된 텍스트 일부 출력

#print(f'type : {type(pages)} / len : {len(pages)} / pages : {pages}')


▹ 미국 백악관 예산관리국, 정부의 책임 있는 AI 조달을 위한 지침 발

위원회에 따르면 연방정부와 법 집행기관에서 얼굴인식 기술이 빠르게 도입되
대통령의 AI 행정명령에 따라 연방정부의 책임 있는 AI 조달을  지원하
르면 AI는 고급 데이터 분석, 디지털 증거 수집, 이미지와 비디오  분
 AI 도입 모범사례와 거버넌스 프레임워크,  정책 옵션을 토대로 공공 
입안자를 대상으로 생성AI의 공익적 활용과 경제사회적  균형 달성, 위험
 2024년 3분기 AI 스타트업은 전체 벤처 투자의 31%를 유치했으며
동영상 제작, 동영상 편집, 오디오 생성과 같은 기능을 지원하는  ‘메타
두 처리할 수 있는 모델과 모바일 기기에서 실행 가능한 경량  모델을 포
모’ 공개 n 앨런AI연구소가 공개한 멀티모달 LLM 제품군 몰모는 벤치
스로 사용할 수 있는 경량 모델 ‘레 미니스트로’를  미스트랄 3B와 미
 맥락 속에서 주요 정보를 기억해 이용자에게 최적화된 답변을 제공하는 A
 바탕으로 인공 신경망을 이용한 머신러닝의 토대가 되는  방법을 개발한 
국 국무부는 바이든 대통령의 AI 행정명령에 따라 국제협력을 통해 포괄적
공자가 안전성 평가에 참조할 수 있는 ‘AI 안전성에 대한  평가 관점 
마인드가 강화학습 방식으로 반도체 칩 레이아웃을 설계하여 사람이 몇 주에
1의 오리 고센 CEO는 AI 모델 개발에 주로 활용되는 트랜스포머  아
터가 설문조사를 통해 근로자 관점의 자동화 기술의 영향을 조사한 결과, 
 2025년 이직을 계획 중이며, 58%는 2024년 중 현재보다  더 
따르면 생성AI의 도입으로 중장기적으로 소프트웨어 엔지니어링에서 데이터 
에 대한 생성AI의 수행 능력을 분석해 인간을 대체할  가능성을 평가한 
리 법인으로 매년 학제간  학술대회(NeurIPS)를 주최 - 이번 제3
 Seongnam-si, Gyeonggi-do, Republic of K


### 03_TextSplitter

In [21]:
# CharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

# 텍스트를 두 줄바꿈("\n\n")을 기준으로 분할, 최대 500자, 50자 겹침, 정규식 아닌 일반 문자열로 처리
text_splitter = CharacterTextSplitter(
    separator="\n\n",  # 텍스트를 두 줄바꿈 기준으로 분할
    chunk_size=500,    # 각 덩어리의 최대 크기 500자
    chunk_overlap=50,  # 각 덩어리 간 50자 겹침
    length_function=len,  # 텍스트 길이 계산에 `len()` 사용
    is_separator_regex=False  # separator는 정규식이 아닌 일반 문자열로 처리
)

splits_CTS = text_splitter.split_documents(pages)

# Document 객체의 page_content 속성에서 텍스트의 길이를 측정
char_list = []
for i in range(len(splits_CTS)):
    char_list.append(len(splits_CTS[i].page_content))
print("chunk_size:")
print(char_list[:20])
print("="*100)

# 상위 50개 청크 선택
top_50_chunks_CTS = splits_CTS[:5]  # 순차적으로 상위 50개 청크 선택(제출용 5개)

# 상위 50개 청크 출력
for i, chunk in enumerate(top_50_chunks_CTS):
    print(f"Chunk {i + 1}: {chunk}")

chunk_size:
[10, 1084, 17, 1542, 1567, 1547, 1523, 1568, 1488, 1451, 1673, 1378, 1351, 1460, 1727, 1587, 1594, 1406, 1620, 1430]
Chunk 1: page_content='2024년 11월호'
Chunk 2: page_content='2024년 11월호 Ⅰ. 인공지능 산업 동향 브리프  1. 정책/법제     ▹ 미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석 1    ▹ 미국 백악관 예산관리국, 정부의 책임 있는 AI 조달을 위한 지침 발표 2    ▹ 유로폴, 법 집행에서 AI의 이점과 과제를 다룬 보고서 발간 3    ▹ OECD, 공공 부문의 AI 도입을 위한 G7 툴킷 발표 4    ▹ 세계경제포럼, 생성AI 시대의 거버넌스 프레임워크 제시 5    2. 기업/산업     ▹ CB인사이츠 분석 결과, 2024년 3분기 벤처 투자 31%가 AI 스타트업에 집중 6    ▹ 메타, 동영상 생성AI 도구 ‘메타 무비 젠’ 공개 7    ▹ 메타, 이미지와 텍스트 처리하는 첫 멀티모달 AI 모델 ‘라마 3.2’ 공개 8    ▹ 앨런AI연구소, 벤치마크 평가에서 GPT-4o 능가하는 성능의 오픈소스 LLM ‘몰모’ 공개 9    ▹ 미스트랄AI, 온디바이스용 AI 모델 ‘레 미니스트로’ 공개 10    ▹ 카카오, 통합 AI 브랜드 겸 신규 AI 서비스 ‘카나나’ 공개 11  3. 기술/연구    ▹ 2024년 노벨 물리학상과 화학상, AI 관련 연구자들이 수상 12    ▹ 미국 국무부, AI 연구에서 국제협력을 위한 ‘글로벌 AI 연구 의제’ 발표 13    ▹ 일본 AI안전연구소, AI 안전성에 대한 평가 관점 가이드 발간 14    ▹ 구글 딥마인드, 반도체 칩 레이아웃 설계하는 AI 모델 ‘알파칩’ 발표 15    ▹ AI21 CEO, AI 에이전트에 트랜스포머 아키텍처의 대안 필요성 강조 16      4. 인력/교육     

#### CharacterTextSplitter(비추천)

`CharacterTextSplitter`는 단일 구분자를 기준으로 텍스트를 분리합니다. 주로 하나의 특정 구분자(예: 공백, 구두점 등)를 사용하여 텍스트를 여러 조각으로 나누는 방식입니다. 이 방법은 텍스트가 일정한 패턴이나 규칙을 따를 때 유용하지만, 텍스트의 길이가 너무 길어져서 모델의 최대 토큰 수(맥스 토큰)를 초과할 수 있습니다. 이 경우, 텍스트를 분할할 때 자연스럽지 않거나 부자연스러운 지점에서 끊어질 수 있어, 전체적인 의미나 흐름이 손상될 위험이 존재합니다.

따라서, `CharacterTextSplitter`는 맥스 토큰 수를 유지하려는 목적에서는 일부 한계를 가질 수 있으며, 긴 텍스트를 처리할 때는 그 자체로 문제가 발생할 수 있습니다.

In [22]:
# RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

splits_RCTS = recursive_text_splitter.split_documents(pages)

# Document 객체의 page_content 속성에서 텍스트의 길이를 측정
char_list = []
for i in range(len(splits_RCTS)):
    char_list.append(len(splits_RCTS[i].page_content))
print("chunk_size:")
print(char_list[:20])
print("="*100)

# 상위 50개 청크 선택
top_50_chunks_RCTS = splits_RCTS[:5]  # 순차적으로 상위 50개 청크 선택(제출용 5개)

# 상위 50개 청크 출력
for i, chunk in enumerate(top_50_chunks_RCTS):
    print(f"Chunk {i + 1}: {chunk}")

chunk_size:
[10, 499, 497, 181, 17, 497, 496, 499, 188, 497, 498, 499, 214, 498, 495, 497, 201, 500, 496, 498]
Chunk 1: page_content='2024년 11월호'
Chunk 2: page_content='2024년 11월호 Ⅰ. 인공지능 산업 동향 브리프  1. 정책/법제     ▹ 미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석 1    ▹ 미국 백악관 예산관리국, 정부의 책임 있는 AI 조달을 위한 지침 발표 2    ▹ 유로폴, 법 집행에서 AI의 이점과 과제를 다룬 보고서 발간 3    ▹ OECD, 공공 부문의 AI 도입을 위한 G7 툴킷 발표 4    ▹ 세계경제포럼, 생성AI 시대의 거버넌스 프레임워크 제시 5    2. 기업/산업     ▹ CB인사이츠 분석 결과, 2024년 3분기 벤처 투자 31%가 AI 스타트업에 집중 6    ▹ 메타, 동영상 생성AI 도구 ‘메타 무비 젠’ 공개 7    ▹ 메타, 이미지와 텍스트 처리하는 첫 멀티모달 AI 모델 ‘라마 3.2’ 공개 8    ▹ 앨런AI연구소, 벤치마크 평가에서 GPT-4o 능가하는 성능의 오픈소스 LLM ‘몰모’ 공개 9    ▹ 미스트랄AI, 온디바이스용 AI 모델 ‘레 미니스트로’'
Chunk 3: page_content='LLM ‘몰모’ 공개 9    ▹ 미스트랄AI, 온디바이스용 AI 모델 ‘레 미니스트로’ 공개 10    ▹ 카카오, 통합 AI 브랜드 겸 신규 AI 서비스 ‘카나나’ 공개 11  3. 기술/연구    ▹ 2024년 노벨 물리학상과 화학상, AI 관련 연구자들이 수상 12    ▹ 미국 국무부, AI 연구에서 국제협력을 위한 ‘글로벌 AI 연구 의제’ 발표 13    ▹ 일본 AI안전연구소, AI 안전성에 대한 평가 관점 가이드 발간 14    ▹ 구글 딥마인드, 반도체 칩 레이아웃 설계하는 AI 모델 ‘알파칩’ 발표 15    ▹ AI21 C

#### RecursiveCharacterTextSplitter(추천)

`RecursiveCharacterTextSplitter`는 여러 구분자를 사용하여 텍스트를 재귀적으로 나누는 방법입니다. 이 방식은 하나의 구분자로 텍스트를 나눈 뒤, 그 결과물에서 또 다른 구분자를 사용해 분할을 반복하는 방식입니다. 이 과정을 통해 더 적절한 지점에서 텍스트를 나누어, 최대 토큰 수를 초과하거나 부족한 상황에 맞게 텍스트의 길이를 동적으로 조절할 수 있습니다.

주요 특징은 **맥스 토큰**을 초과할 경우 다른 구분자를 찾아서 분할하고, 반대로 토큰 수가 부족할 경우 자연스럽게 텍스트의 길이를 재조정할 수 있다는 점입니다. 이 과정은 구분자를 재귀적으로 탐색하여 길이를 맞추기 때문에 더 유연하고 효율적으로 텍스트를 다룰 수 있습니다.

따라서, `RecursiveCharacterTextSplitter`는 텍스트의 의미를 최대한 유지하면서도 효율적으로 토큰 길이를 조정할 수 있어, **대규모 텍스트 처리**에서 훨씬 더 안정적이고 신뢰할 수 있는 분할 방법이 됩니다.

In [23]:
# tiktoken 라이브러리와 langchain의 RecursiveCharacterTextSplitter 임포트
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 'cl100k_base' 토크나이저를 가져옴 (OpenAI GPT 모델용)
tokenizer = tiktoken.get_encoding("cl100k_base")

# tiktoken을 사용하여 텍스트의 토큰 길이를 계산하는 함수 정의
def tiktoken_len(text):
    # 입력된 텍스트를 'cl100k_base' 토크나이저로 인코딩하여 토큰으로 변환
    tokens = tokenizer.encode(text)
    # 토큰의 개수를 반환
    return len(tokens)

# 텍스트 분할기 설정
recursive_text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0, length_function = tiktoken_len)
# 페이지를 500자씩 분할하고, 중복되는 텍스트가 없도록 설정
splits_RCTS_tiktoken = recursive_text_splitter.split_documents(pages)  # 분할된 텍스트를 docs 변수에 저장

# Document 객체의 page_content 속성에서 텍스트의 길이를 측정
token_list = []  # 토큰 수를 저장할 리스트
for i in range(len(splits_RCTS_tiktoken)):
    token_list.append(tiktoken_len(splits_RCTS_tiktoken[i].page_content))  # page_content에 대해 토큰 수 계산

print("Token Count (for each chunk):")
print(token_list[:20])  # 처음 20개 토큰 수 출력
print("="*100)

# 상위 50개 청크 선택
top_50_splits_RCTS_tiktoken = splits_RCTS_tiktoken[:5]  # 순차적으로 상위 50개 청크 선택(제출용 5개)

# 상위 50개 청크 출력
for i, chunk in enumerate(top_50_splits_RCTS_tiktoken):
    print(f"Chunk {i + 1}: {chunk}")

Token Count (for each chunk):
[9, 472, 462, 20, 23, 497, 497, 500, 497, 499, 383, 497, 496, 384, 481, 499, 352, 495, 499, 437]
Chunk 1: page_content='2024년 11월호'
Chunk 2: page_content='2024년 11월호 Ⅰ. 인공지능 산업 동향 브리프  1. 정책/법제     ▹ 미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석 1    ▹ 미국 백악관 예산관리국, 정부의 책임 있는 AI 조달을 위한 지침 발표 2    ▹ 유로폴, 법 집행에서 AI의 이점과 과제를 다룬 보고서 발간 3    ▹ OECD, 공공 부문의 AI 도입을 위한 G7 툴킷 발표 4    ▹ 세계경제포럼, 생성AI 시대의 거버넌스 프레임워크 제시 5    2. 기업/산업     ▹ CB인사이츠 분석 결과, 2024년 3분기 벤처 투자 31%가 AI 스타트업에 집중 6    ▹ 메타, 동영상 생성AI 도구 ‘메타 무비 젠’ 공개 7    ▹ 메타, 이미지와 텍스트 처리하는 첫 멀티모달 AI 모델 ‘라마 3.2’ 공개 8    ▹ 앨런AI연구소, 벤치마크 평가에서 GPT-4o 능가하는 성능의 오픈소스 LLM ‘몰모’ 공개 9    ▹ 미스트랄AI, 온디바이스용 AI 모델 ‘레'
Chunk 3: page_content='미니스트로’ 공개 10    ▹ 카카오, 통합 AI 브랜드 겸 신규 AI 서비스 ‘카나나’ 공개 11  3. 기술/연구    ▹ 2024년 노벨 물리학상과 화학상, AI 관련 연구자들이 수상 12    ▹ 미국 국무부, AI 연구에서 국제협력을 위한 ‘글로벌 AI 연구 의제’ 발표 13    ▹ 일본 AI안전연구소, AI 안전성에 대한 평가 관점 가이드 발간 14    ▹ 구글 딥마인드, 반도체 칩 레이아웃 설계하는 AI 모델 ‘알파칩’ 발표 15    ▹ AI21 CEO, AI 에이전트에 트랜스포머 아키텍처의 대안 필요성 강조

#### RecursiveCharacterTextSplitter + 'cl100k_base' tokenizer (OpenAI 최적화)

length_function는 텍스트의 길이를 계산하는 함수입니다. tiktoken_len 함수는 tiktoken을 사용하여 토큰의 개수를 기반으로 길이를 계산합니다.

이 방식의 중요한 특징은, 텍스트의 의미를 유지하면서도 효율적으로 토큰 수를 조정할 수 있다는 점입니다. 예를 들어, 각 문서나 텍스트가 일정한 크기(토큰 수)에 맞게 나눠져야 하는데, 이때 RecursiveCharacterTextSplitter는 의미 단위에서 자연스럽게 분할점을 찾을 수 있게 도와줍니다. 대규모 데이터나 문서에서 토큰 길이를 최적화하며 분할할 때 유용합니다. 예를 들어, GPT 모델의 입력 제한을 고려하면서 텍스트를 의미 있게 분할할 수 있습니다.

### 04_OpenAIEmbeddings

In [24]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


# 임베딩 결과를 캐시해서 저장, 과금 절약 CacheBackedEmbeddings
store = LocalFileStore("./cache/")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(embeddings, store, namespace=embeddings.model)

# 각 분할된 텍스트에 대해 임베딩 생성
doc_list = [split.page_content for split in splits_RCTS_tiktoken]  # 텍스트만 추출
embeddings_list = embeddings.embed_documents(doc_list)


# 임베딩 결과 출력 (예시)
for idx, embedding in enumerate(embeddings_list[:5]):  # 첫 5개만 출력 예시
    print(f"Embedding {idx + 1}: {embedding[:10]}...")  # 임베딩의 앞부분만 출력

Embedding 1: [-0.01237500924617052, -0.017598269507288933, 0.00840690080076456, -0.022017449140548706, -0.01361054927110672, 0.024763094261288643, -0.029234571382403374, 0.0054161096923053265, -0.029652955010533333, 0.030855808407068253]...
Embedding 2: [0.0010611381148919463, -0.04015206918120384, 0.007658143062144518, -0.024157719686627388, 0.001357791479676962, 0.0025244615972042084, -0.030340073630213737, 0.025114987045526505, -0.022269774228334427, 0.004028502386063337]...
Embedding 3: [-0.009871292859315872, -0.035262301564216614, 0.018142199143767357, -0.02494720369577408, -0.006596551742404699, 0.013919329270720482, -0.029882850125432014, 0.03467056155204773, -0.010570621117949486, 0.01168013270944357]...
Embedding 4: [-0.013028128072619438, -0.012120729312300682, 0.023049255833029747, 0.006063676439225674, 0.0022453153505921364, 0.01881031133234501, -0.03324921429157257, -0.008133472874760628, -0.01582980528473854, -0.003967386204749346]...
Embedding 5: [-0.009894178248941898,

#### CacheBackedEmbeddings
**과금 절약**: 이 방식은 주로 클라우드에서 API 호출 시 비용이 발생하는 경우, 이미 계산된 임베딩을 로컬 캐시에 저장하여, 반복된 계산을 피하고 네트워크 요청을 줄여 과금을 절약하는 데 유용합니다.

**로컬 캐시 활용**: LocalFileStore를 사용해 캐시를 로컬에 저장하므로, 같은 데이터에 대해 다시 계산할 필요가 없어 성능을 향상시키고 리소스를 절약할 수 있습니다.

### 05_Vectorstores

In [25]:
import chromadb
from langchain_community.vectorstores import Chroma, FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# Chroma 클라이언트 설정
client = chromadb.Client()

# FAISS VectorStore 생성
faiss_vectorstore = FAISS.from_documents(documents=splits_RCTS_tiktoken, embedding=embeddings)

# Chroma 클라이언트 설정
client = chromadb.Client()
chroma_vectorstore = Chroma.from_documents(documents=splits_RCTS_tiktoken, embedding=embeddings, client=client)

# BM25 retriever 생성
bm25_retriever = BM25Retriever.from_texts(doc_list)
bm25_retriever.k = 3

# Chroma retriever 생성
chroma_retriever = chroma_vectorstore.as_retriever(search_kwargs={"k": 3})

# FAISS retriever 생성
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 3})

# EnsembleRetriever 생성 (FAISS, Chroma, 결과 결합)
ensemble_01_retriever = EnsembleRetriever(
    retrievers=[faiss_retriever, chroma_retriever],
    weight=[0.5, 0.5]
)

# EnsembleRetriever 생성 (FAISS, BM25 결과 결합)
ensemble_02_retriever = EnsembleRetriever(
    retrievers=[faiss_retriever, bm25_retriever],
    weight=[0.5, 0.5]
)

# EnsembleRetriever 생성 (Chroma, BM25 결과 결합)
ensemble_03_retriever = EnsembleRetriever(
    retrievers=[chroma_retriever, bm25_retriever],
    weight=[0.5, 0.5]
)

# EnsembleRetriever 생성 (Chroma, FAISS, BM25 결과 결합)
ensemble_04_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever, faiss_retriever],
    weight=[0.33, 0.33, 0.33]  # 세 가지 retriever의 결과를 동일한 비율로 결합
)

### 05_01_Vectorstores_Output(생략)

In [26]:
# 검색 결과 문서를 가져옵니다.
query = "메타 인공지능"

faiss_result = faiss_retriever.invoke(query)
chroma_result = chroma_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
ensemble_01_result = ensemble_01_retriever.invoke(query)
ensemble_02_result = ensemble_02_retriever.invoke(query)
ensemble_03_result = ensemble_03_retriever.invoke(query)
ensemble_04_result = ensemble_04_retriever.invoke(query)

print("[FAISS Retriever]")
for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

print("[chroma_retriever]")
for doc in chroma_result:
    print(f"Content: {doc.page_content}")
    print()

print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

# 가져온 문서를 출력합니다.
print("[FAISS, Chroma Ensemble Retriever]")
for doc in ensemble_01_result:
    print(f"Content: {doc.page_content}")
    print()

print("[FAISS, BM25 Ensemble Retriever]")
for doc in ensemble_02_result:
    print(f"Content: {doc.page_content}")
    print()

print("[Chroma, BM25 Ensemble Retriever]")
for doc in ensemble_03_result:
    print(f"Content: {doc.page_content}")
    print()

print("[Chroma, FAISS, BM25 Ensemble Retriever]")
for doc in ensemble_04_result:
    print(f"Content: {doc.page_content}")
    print()

[FAISS Retriever]
Content: 아키텍처이지만, 다중 에이전트 생태계 조성  측면에서는 한계를 내포 ∙트랜스포머 아키텍처는 처리하는 컨텍스트가 길수록 속도가 느리고 연산 비용이 많이 드는데, AI  에이전트는 LLM을 여러 차례 호출해야 하고 각 단계에서 광범위한 컨텍스트를 사용하는 경우가 많아  처리 과정에서 지연이 발생 n 고센 CEO는 ‘맘바(Mamba)’와 ‘잠바(Jamba)’와 같은 대체 아키텍처를 활용하면 AI 에이전트를 더  효율적이고 저렴하게 만들 수 있다고 강조 ∙카네기멜론⼤와 프린스턴⼤ 연구진이 개발한 맘바는 트랜스포머 모델의 핵심인 어텐션(Attention)*  메커니즘 대신 데이터를 우선순위에 따라 정리하고 입력에 가중치를 부여해 메모리 사용을 최적화 * 입력된 데이터 간 연관성을 파악해 상호작용을 계산하는 메커니즘  ∙미스트랄이 2024년 7월 ‘코드스트랄(Codestral) 맘바 7B’를, UAE의 AI 기업 팔콘(Falcon)이 8월  ‘팔콘 맘바 7B’를 출시하는 등, 최근 오픈소스 AI 개발자 사이에서 맘바의 인기가 높아지는 추세 ∙AI21 역시 맘바

Content: 1. 정책/법제   2. 기업/산업  3. 기술/연구   4. 인력/교육 7 메타, 동영상 생성AI 도구 ‘메타 무비 젠’ 공개 n 메타가 동영상 생성, 개인화 동영상 제작, 동영상 편집, 오디오 생성과 같은 기능을 지원하는  ‘메타 무비 젠’을 공개하고 2025년 중 인스타그램 등 자사 플랫폼에 통합할 계획 n 메타 무비 젠은 인간 선호도 평가에서 런웨이의 젠 3, 오픈AI의 소라, 클링 1.5와 같은 경쟁  동영상 AI 모델보다 더 높은 점수를 기록 KEY Contents £ 메타, 동영상 제작과 편집, 오디오 생성을 지원하는 메타 무비 젠을 공개  n 메타(Meta)가 2024년 10월 4일 텍스트 입력을 통해 고해상도 동영상을 생성하는 AI 도구 ‘메타  무비 젠(Meta Movie Gen)’을 공개 ∙메타는 크리에이터와 영화 제작자 등 소수

#### 단독

**FAISS** : 문서의 임베딩 벡터를 사용하여 의미적 유사성을 기반으로 검색합니다. 즉, 문서의 의미와 관련된 유사한 벡터를 찾아냅니다. FAISS는 대규모 데이터셋에서 고속으로 검색할 수 있는 효율적인 방법입니다.

**Chroma**: 벡터 기반의 의미적 유사성 검색을 수행하게 됩니다. Chroma는 임베딩 벡터를 사용하여 문서 간의 의미적 유사성을 계산하고, 쿼리와 가장 유사한 문서를 찾아 반환합니다.

**BM25 Retriever** : 주로 정확한 단어 일치를 기반으로 하여, 쿼리와 문서의 키워드 일치 정도에 따라 순위를 매깁니다.

#### 앙상블

**FAISS + Chroma** : FAISS는 벡터 기반의 의미적 유사성, Chroma는 벡터와 데이터베이스를 사용하여 보다 다양한 의미적 연관성을 찾아냅니다.

**FAISS + BM25** : FAISS의 벡터 기반 검색과 BM25의 키워드 기반 검색을 결합하여, 두 방식의 장점을 동시에 취합니다.

**Chroma + BM25** : 텍스트의 의미와 키워드를 동시에 고려한 결과를 제공합니다.(**사용**)

**FAISS + Chroma + BM25** : FAISS는 의미적 유사성, BM25는 키워드 일치, Chroma는 벡터화된 문서 간 유사성에 기반한 검색을 결합하여, 각 검색 방식이 보완적으로 작용하게 됩니다.


### 06_LLM_model

In [27]:
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler # 스트리밍 방식으로 출력
from langchain_core.messages import HumanMessage
# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini", 
                   callbacks=[StreamingStdOutCallbackHandler()], 
                   streaming=True,
                   temperature = 0, # temperature = 0 ~ 2 높을 수록 다양한 답변. 버전에 따라 디폴트 값이 변경될 수 있기 때문에 명시필요.
                   max_tokens = 1000
                  ) 

### 07_LongContextReorder(미사용)

In [28]:
from langchain.chains import LLMChain, StuffDocumentsChain 
from langchain_community.document_transformers import LongContextReorder 

reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(ensemble_03_retriever)

### 08_RAG+prompt_engineering_load_and_save 

In [56]:
import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt_file = "prompt_002"

# system_prompt를 /01_prompts/prompt_000.txt 파일에서 읽어오기
def load_system_prompt():
    with open(f"01_prompts/{prompt_file}.txt", "r", encoding="utf-8") as file:
        return file.read().replace("\n", " ")  # \n을 공백으로 대체
        
# 시스템 프롬프트 로드
system_prompt = load_system_prompt()

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        
        # output의 실제 타입 확인
        #print("Type of output:", type(output))
        #print("Output content:", output)  # output 내용을 확인

        # 만약 output이 dict 형태라면
        if isinstance(output, dict):
            # 문서 내용 출력 (각 문서 구분만 하기)
            documents = output.get('context', [])
            for idx, doc in enumerate(documents):
                # 각 문서의 page_content를 출력
                print(f"Document {idx + 1}:")
                print(doc.page_content)  # Document 객체의 page_content 출력
                print("========================")  # 각 문서 구분선
        else:
            # 만약 output이 문자열이라면, 그냥 출력
            #print("Received output is not a dictionary, but a string:")
            print(output)
        
        return output

# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": ensemble_03_retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()| contextual_prompt | model

# 출력 결과와 질문을 /02_results/ 폴더에 저장하는 함수
def save_output_to_file(question, response):
    # 현재 시간을 timestamp 형식으로 기록
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # 결과를 저장할 파일 경로
    output_filename = f"02_results/{prompt_file}_{question}.txt"
    
    # 파일을 생성하고 저장
    with open(output_filename, "w", encoding="utf-8") as file:
        file.write(f"시스템: {system_prompt}\n\n")
        file.write(f"질문: {question}\n\n")
        file.write(f"답변: {response.content}\n\n")
        file.write(f"타임스탬프: {timestamp}\n")

### 09_RAG_답변

In [57]:
while True:
    # query = input("질문을 입력하세요: ")
    query = "메타의 인공지능에 대해 알려줘"
    #print("========================")
    response = rag_chain_debug.invoke(query)
    print()
    print()
    print("Final Response")
    print()
    print("시스템:", system_prompt,"\n")
    print("질문:", query,"\n")
    print("딥변:", response.content,"\n")
    save_output_to_file(query, response)# 결과를 /02_results/ 폴더에 저장
    break

메타의 인공지능에 대해 알려줘
Document 1:
아키텍처이지만, 다중 에이전트 생태계 조성  측면에서는 한계를 내포 ∙트랜스포머 아키텍처는 처리하는 컨텍스트가 길수록 속도가 느리고 연산 비용이 많이 드는데, AI  에이전트는 LLM을 여러 차례 호출해야 하고 각 단계에서 광범위한 컨텍스트를 사용하는 경우가 많아  처리 과정에서 지연이 발생 n 고센 CEO는 ‘맘바(Mamba)’와 ‘잠바(Jamba)’와 같은 대체 아키텍처를 활용하면 AI 에이전트를 더  효율적이고 저렴하게 만들 수 있다고 강조 ∙카네기멜론⼤와 프린스턴⼤ 연구진이 개발한 맘바는 트랜스포머 모델의 핵심인 어텐션(Attention)*  메커니즘 대신 데이터를 우선순위에 따라 정리하고 입력에 가중치를 부여해 메모리 사용을 최적화 * 입력된 데이터 간 연관성을 파악해 상호작용을 계산하는 메커니즘  ∙미스트랄이 2024년 7월 ‘코드스트랄(Codestral) 맘바 7B’를, UAE의 AI 기업 팔콘(Falcon)이 8월  ‘팔콘 맘바 7B’를 출시하는 등, 최근 오픈소스 AI 개발자 사이에서 맘바의 인기가 높아지는 추세 ∙AI21 역시 맘바
Document 2:
없이 개인정보보호가 가능한 온디바이스 추론을 원하는 고객 수요에 맞게  지연시간이 짧고 효율적인 솔루션을 제공한다고 강조 ∙미스트랄AI는 8B 버전만 연구용으로 다운로드를 허용했으며 향후 두 모델을 클라우드 플랫폼을 통해  제공할 계획으로, 사용 비용은 100만 입출력 토큰 당 8B 버전은 10센트, 3B 버전은 4센트로 책정 £ 레 미니스트로, 오픈소스 모델 ‘젬마’ 및 ‘라마’ 대비 대부분 벤치마크에서 우수한 평가 n 벤치마크 평가 결과, 레 미니스트로는 비슷한 매개변수를 가진 오픈소스 모델 젬마(Gemma)와 라마 (Llama)보다 대부분 벤치마크에서 더 높은 평가를 획득 ∙MMLU* 평가에서 미스트랄 3B는 60.9점을 얻어 구글의 ‘젬마 2 2B’(52.4점)와 메타의 ‘라마 3.2  3B’(56.2점)를 앞섰고, 미스트랄 8B는 6

### 09_일반LLM(gpt-4o-mini)_답변

In [58]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# OpenAI 클라이언트 설정
client = OpenAI(api_key=api_key)

# 사용자 질문에 대한 completion 생성
completion = client.chat.completions.create(
    model='gpt-4o-mini',  # 사용하려는 모델
    messages=[{'role': 'user', 'content': query}],
    temperature=0.0  # 정확한 응답을 원할 때 사용
)

# 응답 출력
print(completion.choices[0].message.content)


메타(Meta, 이전의 페이스북)는 인공지능(AI) 기술을 다양한 분야에 활용하고 있습니다. 메타의 AI 연구는 주로 다음과 같은 영역에서 진행되고 있습니다:

1. **자연어 처리(NLP)**: 메타는 자연어 처리 기술을 통해 사용자와의 상호작용을 개선하고, 콘텐츠 추천 시스템을 발전시키고 있습니다. 예를 들어, 메신저와 같은 플랫폼에서 사용자 간의 대화를 이해하고, 적절한 응답을 생성하는 데 AI를 활용합니다.

2. **컴퓨터 비전**: 이미지와 비디오 분석을 위한 AI 기술도 개발하고 있습니다. 이는 사진 태그, 콘텐츠 필터링, 증강 현실(AR) 기능 등 다양한 응용 프로그램에 사용됩니다.

3. **추천 시스템**: 메타는 사용자에게 맞춤형 콘텐츠를 제공하기 위해 AI 기반의 추천 알고리즘을 사용합니다. 이는 사용자 행동 데이터를 분석하여 개인화된 피드를 제공하는 데 기여합니다.

4. **가상 현실(VR) 및 증강 현실(AR)**: 메타는 Oculus와 같은 VR 기기를 통해 몰입형 경험을 제공하며, AI를 활용하여 사용자 경험을 향상시키고 있습니다. 예를 들어, 가상 환경에서의 상호작용을 보다 자연스럽게 만드는 데 AI 기술이 사용됩니다.

5. **안전 및 보안**: 메타는 AI를 사용하여 플랫폼에서의 부적절한 콘텐츠를 감지하고 차단하는 시스템을 개발하고 있습니다. 이는 사용자 안전을 보장하고 커뮤니티 가이드라인을 준수하는 데 중요한 역할을 합니다.

메타는 AI 연구와 개발에 많은 자원을 투자하고 있으며, 이를 통해 사용자 경험을 개선하고, 새로운 기술을 선보이는 데 주력하고 있습니다. AI 기술의 발전은 메타의 비즈니스 모델과 서비스에 큰 영향을 미치고 있습니다.


### 랭체인을 사용하는 이유: 최신 정보 반영의 중요성

랭체인(Chain)은 **일반적인 AI 모델**보다 **실시간 데이터와 최신 정보를 반영**할 수 있다는 점에서 유리합니다. 이는 여러 가지 이유로 중요합니다:

1. **최신 정보 반영**  
   일반적인 AI 모델은 고정된 데이터셋을 기반으로 학습되기 때문에, **새로운 정보나 실시간 데이터를 반영하는 데 한계**가 있을 수 있습니다. 예를 들어, 최신 기술 발표나 업데이트된 벤치마크 결과 같은 정보는 AI 모델이 학습 후에는 반영되지 않거나 오래된 데이터일 수 있습니다.

2. **랭체인의 실시간 데이터 처리**  
   반면, **랭체인**은 **외부 데이터 소스와 실시간으로 연결**되어 정보를 즉시 가져오고 처리할 수 있습니다. 즉, 사용자가 요구하는 질문에 대해 **최신 뉴스**나 **업데이트된 정보를 실시간으로 반영**하여 보다 **정확하고 신뢰할 수 있는 답변**을 제공할 수 있습니다.

3. **정확성 강화**  
   예를 들어, **메타의 메타 무비 젠**과 같은 **최신 기술 발표**나 **최신 AI 모델 성능**에 대한 정보는 랭체인을 통해 즉시 반영됩니다. 이는 **일반 AI 모델**이 **학습하지 못한 최신 정보**를 제공함으로써, 보다 **정확하고 실시간적인 답변**을 가능하게 합니다.